In [1]:
!pip install oandapyV20

import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades 
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.transactions as trans
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.definitions.primitives as primitives
import pandas as pd
from dateutil.relativedelta import relativedelta
import pandas as pd


import os
import io
from io import StringIO 
import boto3
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import time 

#import datedelta
import calendar
import  csv
import json

from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import numpy as np
import datetime
from scipy import stats
def convert_to_df(instrument,granularity = "H1",count = 5000,percentile_test = 97,
                  test_col = 'delta_percentile_prev_wick',positive = True,start = date(2020,1,1),end = date(2020,1,10)):
    print()
    print()
    print()
    print(instrument,granularity,count,percentile_test)
    print('START:',start,' END:',end)
    if positive:
        negative = False
    else:
        negative = True
    params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }
    print('PARAMS',params)
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:
        #IF CANDLE IS BULLISH
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        #IF CANDLE IS BEARISH
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
            prev_wick = 0
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            
            if (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o'])) > 0:
                prev_wick = float(prev_candle['mid']['h']) - float(prev_candle['mid']['c'])
            else:
                prev_wick = float(prev_candle['mid']['l']) - float(prev_candle['mid']['o'])


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector,prev_wick

                   ])
        
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector','prev_wick'])
    
    
    df['prev_5'] = 0
    df['prev_10'] = 0
    df['prev_24'] = 0
    
    
    df['next_candle'] = 0
    df['next_2'] = 0
    df['next_3'] = 0
    df['next_4'] = 0
    df['next_5'] = 0
    df['next_10'] = 0
    
    df['next_candle+1'] = 0
    df['next_2+1'] = 0
    df['next_3+1'] = 0
    df['next_4+1'] = 0
    df['next_5+1'] = 0
    df['next_10+1'] = 0
    df['next_20+1'] = 0
        
    tot_length = df.shape[0]
    for i in range(df.shape[0] - 21):
        
        df['prev_5'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 5] 
        df['prev_10'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 10] 
        df['prev_24'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 24]    
        
        df['next_candle'].iloc[i] = df['c'].iloc[i + 1] - df['c'].iloc[i]
        df['next_2'].iloc[i] = df['c'].iloc[i + 2] - df['c'].iloc[i]
        df['next_3'].iloc[i] = df['c'].iloc[i + 3] - df['c'].iloc[i]
        df['next_4'].iloc[i] = df['c'].iloc[i + 4] - df['c'].iloc[i]
        df['next_5'].iloc[i] = df['c'].iloc[i + 5] - df['c'].iloc[i]
        df['next_10'].iloc[i] = df['c'].iloc[i + 10] - df['c'].iloc[i]
        
        df['next_candle+1'].iloc[i] = df['c'].iloc[i + 1+1] - df['c'].iloc[i+1]
        df['next_2+1'].iloc[i] = df['c'].iloc[i + 2+1] - df['c'].iloc[i+1]
        df['next_3+1'].iloc[i] = df['c'].iloc[i + 3+1] - df['c'].iloc[i+1]
        df['next_4+1'].iloc[i] = df['c'].iloc[i + 4+1] - df['c'].iloc[i+1]
        df['next_5+1'].iloc[i] = df['c'].iloc[i + 5+1] - df['c'].iloc[i+1]
        df['next_10+1'].iloc[i] = df['c'].iloc[i + 10+1] - df['c'].iloc[i+1]
        df['next_20+1'].iloc[i] = df['c'].iloc[i + 20+1] - df['c'].iloc[i+1] 
        
        
    col = test_col
    filter_df = df[df[col] >= np.percentile(df[col], percentile_test)]
    print('SHAPE',filter_df.shape)
    df['trigger'] = 0

    df.loc[df[df[col] >= np.percentile(df[col], percentile_test)].index,'trigger'] = 1    
    
    print('SHAPE2',df[df.trigger == 1].shape)
        
    return df

def convert_to_simple_df(instrument,granularity = "H1",count = 5000,start = date(2020,1,1),end = date(2020,1,10)):
    print()
    print()
    print()
    print(instrument,granularity,count)
    print('START:',start,' END:',end)

    params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }
    print('PARAMS',params)
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:

        lst.append([candle['time'],
                    candle['volume'],
                    float(candle['mid']['o']),
                    float(candle['mid']['h']),
                    float(candle['mid']['l']),
                    float(candle['mid']['c'])
                   ])
        
 
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c'])
    
    df['utc_timestamp'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
    df['est_timestamp'] = df['utc_timestamp'].dt.tz_convert('US/Eastern')        
    return df


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
def convert_to_simple_df(instrument,granularity = "H1",count = 5000,start = date(2020,1,1),end = date(2020,1,10)):
    print()
    print()
    print()
    print(instrument,granularity,count)
    print('START:',start,' END:',end)

    params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }
    print('PARAMS',params)
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    return dct
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:

        lst.append([candle['time'],
                    candle['volume'],
                    float(candle['mid']['o']),
                    float(candle['mid']['h']),
                    float(candle['mid']['l']),
                    float(candle['mid']['c'])
                   ])
        
 
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c'])
    
    df['utc_timestamp'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
    df['est_timestamp'] = df['utc_timestamp'].dt.tz_convert('US/Eastern')        
    return df

#instrument = 'EUR_USD'
#start = datetime.datetime(2020,1,1,0,0,0)
#end = datetime.datetime(2020,1,2,0,0,0)
#dct = convert_to_simple_df(instrument,granularity = "H1",count = 5000,start = start,end = end)
#dct

In [3]:
def convert_to_simple_df(instrument,granularity = "H1",count = 5000,start = date(2020,1,1),end = date(2020,1,10)):
    print()
    print()
    print()
    print(instrument,granularity,count)
    print('START:',start,' END:',end)

    params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }
    print('PARAMS',params)
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:

        lst.append([candle['time'],
                    candle['volume'],
                    float(candle['mid']['o']),
                    float(candle['mid']['h']),
                    float(candle['mid']['l']),
                    float(candle['mid']['c'])
                   ])
        
 
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c'])
    
    df['utc_timestamp'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
    df['est_timestamp'] = df['utc_timestamp'].dt.tz_convert('US/Eastern')        
    return df


def create_data_and_save(instrument,start,end,pair,hours = 3,granularity = "M1"):

    print('CREATING DATASET...')
    s = time.time()
    print('PARAMS:',start,end,pair,granularity)
    cur = start

    #i = 'GBP_JPY'
    total_df = 0
    dct = {}
    
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    while cur < end:
        print(cur)    
        df = convert_to_simple_df(instrument = instrument,granularity = granularity,count = 5000,start = cur,end = cur + timedelta(hours = hours))
        

        cur = cur + timedelta(hours = hours)

        print(df.shape)
        if df.shape[1] == 1:
            pass
        else:
            try:
                total_df = total_df.append(df)
            except:
                total_df = df
                
            dct[str(cur)] = df

    print(total_df.shape)
    total_df.to_csv(path,index = False)
    e = time.time()
    print('TOTAL DATASET CONSTRUCTION TIME:',(e-s)/60,' MINUTES')
accountID='101-001-20062555-002'
access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'

api = API(access_token)
client=API(access_token)

jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']    
jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['XAU_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 

all_pairs = ['XAU_USD']
all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 
start = datetime.datetime(2005,1,1,0,0,0)
end = datetime.datetime(2023,1,1,0,0,0)
for pair in all_pairs:
    create_data_and_save(instrument = pair,start = start,end = end,pair = pair,hours = 1000,granularity = "H1")

        

In [4]:
from datetime import datetime, date, timedelta
def create_data_and_save(instrument,start,end,pair,hours = 3,granularity = "M1"):

    print('CREATING DATASET...')
    s = time.time()
    print('PARAMS:',start,end,pair,granularity)
    cur = start

    #i = 'GBP_JPY'
    total_df = 0
    dct = {}
    
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    while cur < end:
        print(cur)    
        df = convert_to_simple_df(instrument = instrument,granularity = granularity,count = 5000,start = cur,end = cur + timedelta(hours = hours))
        

        cur = cur + timedelta(hours = hours)

        print(df.shape)
        if df.shape[1] == 1:
            pass
        else:
            try:
                total_df = total_df.append(df)
            except:
                total_df = df
                
            dct[str(cur)] = df

    print(total_df.shape)
    total_df.to_csv(path,index = False)
    e = time.time()
    print('TOTAL DATASET CONSTRUCTION TIME:',(e-s)/60,' MINUTES')
    
    
def smma(df,period = 14):
    print('GETTING SMMA INDICATOR FOR:',period)
    close_col = df.columns.get_loc('c')
    df['smma_' + str(period)] = df['c']
    smma_col = df.columns.get_loc('smma_' + str(period))
    arr = df.values
    
    #SMMA CALC:
    
    #SUM1=SUM (CLOSE, N)

    #SMMA1 = SUM1/ N

    #The second and subsequent moving averages are calculated according to this formula:

    #SMMA (i) = (SUM1 – SMMA1+CLOSE (i))/ N    
    
   # Where:

    #SUM1 – is the total sum of closing prices for N periods;
    #SMMA1 – is the smoothed moving average of the first bar;
    #SMMA (i) – is the smoothed moving average of the current bar (except the first one);
    #CLOSE (i) – is the current closing price;
    #N – is the smoothing period.    

    for i in range(period,arr.shape[0]):
        if i == period:
            sum1 = sum(arr[:i,close_col])

            smma1 = sum1 / period
 
            arr[i,smma_col] = smma1
        elif i == period + 1:
            arr[i,smma_col] = (smma1 * (period - 1) + arr[i,close_col]) / period
            
        else:

            prev_sum = arr[i - 1,smma_col] * period
            arr[i,smma_col] = (prev_sum - arr[i - 1,smma_col] + arr[i,close_col]) / period
        
        
    return pd.DataFrame(arr,columns = df.columns)

def ema(df,num = 14):
    print('GETTING EMA INDICATOR FOR:',num)
    close_col = df.columns.get_loc('c')
    df['ema_' + str(num)] = df['c']
    arr = df.values
    mult = 2/ (num + 1)
    for i in range(num,arr.shape[0]):
        sma = sum(arr[i - num + 1: i + 1,close_col]) / num
        arr[i,-1] = ((arr[i,close_col] - arr[i - 1,-1]) * mult) + arr[i - 1,-1]
    return pd.DataFrame(arr,columns = df.columns)

def engulfing_candle(df):
    print('GETTING ENGULFING CANDLES...')
    df['bearish_engulfing'] = 0
    df['bullish_engulfing'] = 0
    col1 = df.columns.get_loc('bearish_engulfing')
    col2 = df.columns.get_loc('bullish_engulfing')
    
    open_col = df.columns.get_loc('o')
    close_col = df.columns.get_loc('c')
    arr = df.values 
    for i in range(arr.shape[0]):
        obp = arr[i-1,open_col] #open[1]
        cbp = arr[i-1,close_col] #close[1]
        obc = arr[i,open_col] #open
        cbc = arr[i,close_col] #close        
        #If current bar open is less than equal to the previous bar close AND current bar open is less than previous bar open AND current bar close is greater than previous bar open THEN True
        if obc <= cbp and obc < obp and cbc > obp:
            arr[i,col2] = 1
        
        #If current bar open is greater than equal to previous bar close AND current bar open is greater than previous bar open AND current bar close is less than previous bar open THEN True
        elif obc >= cbp and obc > obp and cbc < obp:    
            arr[i,col1] = 1
        
    return pd.DataFrame(arr,columns = df.columns)


def three_line_strike(df):
    print('GETTING 3 LINE STRIKE...')
    df['bearish_tls'] = 0
    df['bullish_tls'] = 0
    col1 = df.columns.get_loc('bearish_tls')
    col2 = df.columns.get_loc('bullish_tls')
    open_col = df.columns.get_loc('o')
    close_col = df.columns.get_loc('c')
    arr = df.values 
    for i in range(4,arr.shape[0]):    
        if arr[i - 3,close_col] > arr[i - 3,open_col] and arr[i - 2,close_col] > arr[i - 2,open_col] and arr[i - 1,close_col] > arr[i - 1,open_col] and arr[i,close_col] < arr[i - 1,open_col]: 
            arr[i,col1] = 1
        if arr[i - 3,close_col] < arr[i - 3,open_col] and arr[i - 2,close_col] < arr[i - 2,open_col] and arr[i - 1,close_col] < arr[i - 1,open_col] and arr[i,close_col] > arr[i - 1,open_col]: 
            arr[i,col2] = 1 
            
    return pd.DataFrame(arr,columns = df.columns)

def convert_timestamp(df):
    time_col = df.columns.get_loc('time')
    arr = df.values 
    for i in range(arr.shape[0]):
        arr[i,time_col] = datetime.strptime(arr[i,time_col][:-4], '%Y-%m-%dT%H:%M:%S.%f')
    return pd.DataFrame(arr,columns = df.columns)

def get_trading_session(df,london_start = 7,london_end = 16,ny_start = 13,ny_end = 22):
    print('GETTING TRADING SESSIONS')
    print('LONDON TRADING SESSION UTC',london_start,london_end)
    print('NEW YORK TRADING SESSION UTC',ny_start,ny_end)
    df['new_york'] = 0
    df['london'] = 0
    
    
    col1 = df.columns.get_loc('new_york')
    col2 = df.columns.get_loc('london')
    utc = df.columns.get_loc('time')
    arr = df.values 

    for i in range(arr.shape[0]):
        if arr[i,utc].hour >= london_start and arr[i,utc].hour < london_end:
            arr[i,col2] = 1
        if arr[i,utc].hour >= ny_start and arr[i,utc].hour < ny_end:
            arr[i,col1] = 1     
    return pd.DataFrame(arr,columns = df.columns)
def clean_and_save_data(pair = 'EUR_USD'
                    ,granularity = 'M5'
                    ,start = datetime(2016,1,1,0,0,0)
                    ,end = datetime(2022,7,31,0,0,0)):
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    print(df.shape)
    df = smma(df,period = 21)
    df = smma(df,period = 50)
    df = smma(df,period = 200)

    df = ema(df,num = 14 * 3)
    df = ema(df,num = 50 * 3)
    df = ema(df,num = 200 * 3)
    df = engulfing_candle(df)
    df = three_line_strike(df)



    df['o'] = df['o'].astype(float)
    df['h'] = df['h'].astype(float)
    df['l'] = df['l'].astype(float)
    df['c'] = df['c'].astype(float)

    df = convert_timestamp(df)
    df = get_trading_session(df,london_start = 7,london_end = 16,ny_start = 13,ny_end = 22)
    print('SAVING DATA AS:',path)
    df.to_csv(path,index = False)
    print()
jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['XAU_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 

granularity = 'M5'
start = datetime(2005,1,1,0,0,0)
end = datetime(2023,1,1,0,0,0)


for pair in all_pairs:
    print(pair)
    clean_and_save_data(pair = pair,granularity = granularity,start = start,end = end)
    
    

GBP_JPY
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2005-01-01_2023-01-01.csv
(1357040, 8)
GETTING SMMA INDICATOR FOR: 21
GETTING SMMA INDICATOR FOR: 50
GETTING SMMA INDICATOR FOR: 200
GETTING EMA INDICATOR FOR: 42
GETTING EMA INDICATOR FOR: 150
GETTING EMA INDICATOR FOR: 600
GETTING ENGULFING CANDLES...
GETTING 3 LINE STRIKE...
GETTING TRADING SESSIONS
LONDON TRADING SESSION UTC 7 16
NEW YORK TRADING SESSION UTC 13 22
SAVING DATA AS: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2005-01-01_2023-01-01.csv

EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2005-01-01_2023-01-01.csv
(1355531, 8)
GETTING SMMA INDICATOR FOR: 21
GETTING SMMA INDICATOR FOR: 50
GETTING SMMA INDICATOR FOR: 200
GETTING EMA INDICATOR FOR: 42
GETTING EMA INDICATOR FOR: 150
GETTING EMA INDICATOR FOR: 600
GETTING ENGULFING CANDLES...
GETTING 3 LINE STRIKE...
GETTING TRADING SESSIONS
LONDON TRADING SESSION UTC 7 16
NEW YO

In [ ]:
def convert_timestamp(df):
    time_col = df.columns.get_loc('time')
    arr = df.values 
    for i in range(arr.shape[0]):
        arr[i,time_col] = datetime.strptime(arr[i,time_col][:-4], '%Y-%m-%dT%H:%M:%S.%f')
    return pd.DataFrame(arr,columns = df.columns)
df = convert_timestamp(df)

In [ ]:
def smma(df,period = 14):
    print('GETTING SMMA INDICATOR FOR:',period)
    close_col = df.columns.get_loc('c')
    df['smma_' + str(period)] = df['c']
    smma_col = df.columns.get_loc('smma_' + str(period))
    arr = df.values
    
    #SMMA CALC:
    
    #SUM1=SUM (CLOSE, N)

    #SMMA1 = SUM1/ N

    #The second and subsequent moving averages are calculated according to this formula:

    #SMMA (i) = (SUM1 – SMMA1+CLOSE (i))/ N    
    
   # Where:

    #SUM1 – is the total sum of closing prices for N periods;
    #SMMA1 – is the smoothed moving average of the first bar;
    #SMMA (i) – is the smoothed moving average of the current bar (except the first one);
    #CLOSE (i) – is the current closing price;
    #N – is the smoothing period.    

    for i in range(period,arr.shape[0]):
        if i == period:
            sum1 = sum(arr[:i,close_col])

            smma1 = sum1 / period
 
            arr[i,smma_col] = smma1
        elif i == period + 1:
            arr[i,smma_col] = (smma1 * (period - 1) + arr[i,close_col]) / period
            
        else:

            prev_sum = arr[i - 1,smma_col] * period
            arr[i,smma_col] = (prev_sum - arr[i - 1,smma_col] + arr[i,close_col]) / period
        
        
    return pd.DataFrame(arr,columns = df.columns)

def get_trading_session(df,london_start = 7,london_end = 16,ny_start = 13,ny_end = 22):
    print('GETTING TRADING SESSIONS')
    print('LONDON TRADING SESSION UTC',london_start,london_end)
    print('NEW YORK TRADING SESSION UTC',ny_start,ny_end)
    df['new_york'] = 0
    df['london'] = 0
    
    
    col1 = df.columns.get_loc('new_york')
    col2 = df.columns.get_loc('london')
    utc = df.columns.get_loc('time')
    arr = df.values 

    for i in range(arr.shape[0]):
        if arr[i,utc].hour >= london_start and arr[i,utc].hour < london_end:
            arr[i,col2] = 1
        if arr[i,utc].hour >= ny_start and arr[i,utc].hour < ny_end:
            arr[i,col1] = 1     
    return pd.DataFrame(arr,columns = df.columns)
    df = smma(df,period = 21)
    df = smma(df,period = 50)
    df = smma(df,period = 200)
df = get_trading_session(df,london_start = 7,london_end = 16,ny_start = 13,ny_end = 22)